<a href="https://colab.research.google.com/github/marreapato/Forecasting_Paper_XAI/blob/main/Tcrossvalidation_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten

# Assuming you have loaded your time series data from Yahoo Finance
# For demonstration purposes, let's create a dummy time series
# Replace this with loading your actual data
# bvsp_data = load_bvsp_data_from_yahoo()

# Example with dummy data
np.random.seed(42)
dates = pd.date_range(start="2022-01-01", end="2023-01-01", freq="D")
bvsp_data = pd.DataFrame({"Close": np.random.rand(len(dates))}, index=dates)

# Feature scaling
scaler = MinMaxScaler()
bvsp_data_scaled = scaler.fit_transform(bvsp_data)

# Assuming you have a time series array and corresponding labels
X = bvsp_data_scaled[:-1]
y = bvsp_data_scaled[1:]

# Function to create LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=128, activation='relu', input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model



In [12]:
# Time series cross-validation using TimeSeriesSplit
n_splits = 30  # You can adjust the number of splits
tscv = TimeSeriesSplit(n_splits=n_splits)
tscv


TimeSeriesSplit(gap=0, max_train_size=None, n_splits=30, test_size=None)

In [14]:

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Reshape the data for LSTM input
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    # Create and train the LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(input_shape)
    model.fit(X_train, y_train, epochs=300, batch_size=32, verbose=0)

    # Evaluate the model on the test set
    mse = model.evaluate(X_test, y_test, verbose=0)
    print(f'Mean Squared Error for this fold: {mse}')

Mean Squared Error for this fold: 0.09955580532550812
Mean Squared Error for this fold: 0.1293274462223053
Mean Squared Error for this fold: 0.07952554523944855
Mean Squared Error for this fold: 0.11949329823255539


KeyboardInterrupt: 